In [10]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import itertools
from functools import reduce
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from qutip import Bloch

class QuantumTomographyApp:
    def __init__(self):
        self.max_qubits = 10

        # Main sliders
        self.n_qubits_slider = widgets.IntSlider(value=2, min=1, max=self.max_qubits, step=1, description='no. of qubits')
        self.n_shotsX_slider = widgets.IntSlider(value=1000, min=100, max=5000, step=100, description='no. of shots X')
        self.n_shotsY_slider = widgets.IntSlider(value=1000, min=100, max=5000, step=100, description='no. of shots Y')
        self.n_shotsZ_slider = widgets.IntSlider(value=1000, min=100, max=5000, step=100, description='no. of shots Z')

        # Initialize all θ and φ sliders
        self.theta_sliders = []
        self.phi_sliders = []
        for i in range(self.max_qubits):
            self.theta_sliders.append(widgets.FloatSlider(
                value=np.pi/2, min=0, max=np.pi, step=0.01,
                description=f'θ_{i}', readout_format='.2f'))
            self.phi_sliders.append(widgets.FloatSlider(
                value=0, min=0, max=2*np.pi, step=0.01,
                description=f'φ_{i}', readout_format='.2f'))

        self.run_button = widgets.Button(description='Run Experiment', button_style='success')
        self.output = widgets.Output()

        # Display initial sliders
        n_qubits = self.n_qubits_slider.value
        visible_sliders = self.theta_sliders[:n_qubits] + self.phi_sliders[:n_qubits]

        self.ui = widgets.VBox([
            self.n_qubits_slider,
            self.n_shotsX_slider,
            self.n_shotsY_slider,
            self.n_shotsZ_slider,
            widgets.VBox(visible_sliders),
            self.run_button,
            self.output
        ])

        self.run_button.on_click(self.run_experiment)
        self.n_qubits_slider.observe(self.update_sliders, names='value')

    def update_sliders(self, change):
        n_qubits = self.n_qubits_slider.value
        visible_sliders = self.theta_sliders[:n_qubits] + self.phi_sliders[:n_qubits]
        self.ui.children = (
            self.ui.children[:4] +
            (widgets.VBox(visible_sliders),) +
            self.ui.children[5:]
        )

    def display(self):
        display(self.ui)

    def generate_multi_qubit_state(self, thetas, phis, n_qubits):
        single_qubit_states = []
        for i in range(n_qubits):
            theta = thetas[i]
            phi = phis[i]
            state = np.array([
                np.cos(theta / 2),
                np.exp(1j * phi) * np.sin(theta / 2)
            ], dtype=complex)
            single_qubit_states.append(state)
        full_state = reduce(np.kron, single_qubit_states)
        return full_state

    def meas_prob(self, state, meas_op_string):
        pauli_dict = {
            'I': np.array([[1,0],[0,1]]),
            'X': np.array([[0,1],[1,0]]),
            'Y': np.array([[0,-1j],[1j,0]]),
            'Z': np.array([[1,0],[0,-1]])
        }
        meas_op = [pauli_dict[op] for op in meas_op_string]
        M = reduce(np.kron, meas_op)
        proj_M = (np.eye(len(state)) + M)/2
        prob_plus1 = (np.conj(state).T) @ proj_M @ state
        return prob_plus1.real

    def generate_samples(self, state, meas_op_string, n_shotsX, n_shotsY, n_shotsZ):
        n_qubit = len(meas_op_string)
        shots_list = []
        for q_idx in range(n_qubit):
            if meas_op_string[q_idx] == 'X':
                shots_list.append(n_shotsX)
            elif meas_op_string[q_idx] == 'Y':
                shots_list.append(n_shotsY)
            elif meas_op_string[q_idx] == 'Z':
                shots_list.append(n_shotsZ)
        n_shots = min(shots_list)
        prob_plus1 = self.meas_prob(state, meas_op_string)
        return np.random.choice([+1, -1], size=n_shots, p=[prob_plus1, 1 - prob_plus1])

    def fidelity(self, state1, state2):
        return np.abs(np.vdot(state1, state2))**2

    def log_likelihood(self, params, samples, n_qubits):
        thetas = params[:n_qubits]
        phis = params[n_qubits:]
        state = self.generate_multi_qubit_state(thetas, phis, n_qubits)
        log_L = 0
        for setting in itertools.product(['I','X','Y','Z'], repeat=n_qubits):
            if setting != ('I',) * n_qubits:
                prob_plus1 = self.meas_prob(state, setting)
                n_plus1 = np.sum(samples[setting] == +1)
                n_minus1 = np.sum(samples[setting] == -1)
                log_L += n_plus1 * np.log(prob_plus1 + 1e-10) + n_minus1 * np.log(1 - prob_plus1 + 1e-10)
        return -log_L

    def estimate_parameters(self, n_qubits, samples):
        opt_val = []
        opt_params = []
        for _ in range(20):
            theta_init = np.random.uniform(0, np.pi, size=n_qubits)
            phi_init = np.random.uniform(0, 2*np.pi, size=n_qubits)
            initial_params = np.concatenate([theta_init, phi_init])
            bounds = [(0, np.pi)] * n_qubits + [(0, 2*np.pi - 0.01)] * n_qubits
            result = minimize(self.log_likelihood, initial_params, args=(samples, n_qubits), bounds=bounds, method='L-BFGS-B')
            opt_val.append(result.fun)
            opt_params.append(result.x)
        best_index = np.argmin(opt_val)
        return opt_params[best_index]

    def plot_thetas_phis(self, true_thetas, true_phis, est_thetas, est_phis, fidelity):
        n_params = len(true_thetas)
        fig = plt.figure(figsize=(max(10, n_params*1.5), 5))
        gs = fig.add_gridspec(2, 1, height_ratios=[1, 4], hspace=0.4)

        ax_fid = fig.add_subplot(gs[0])
        ax_fid.set_xlim(0, 1)
        ax_fid.set_ylim(0, 1)
        ax_fid.axis('off')
        ax_fid.add_patch(plt.Rectangle((0, 0.4), 1, 0.2, color='lightgray'))
        ax_fid.add_patch(plt.Rectangle((0, 0.4), fidelity, 0.2, color='red'))
        ax_fid.text(0.5, 0.8, f"Fidelity: {fidelity:.4f}", ha='center', fontsize=16)

        ax = fig.add_subplot(gs[1])
        labels, values, colors = [], [], []
        for i in range(n_params):
            labels += [f'θ_{i} True', f'θ_{i} Est']
            values += [true_thetas[i], est_thetas[i]]
            colors += ['green', 'lightgreen']
        for i in range(n_params):
            labels += [f'φ_{i} True', f'φ_{i} Est']
            values += [true_phis[i], est_phis[i]]
            colors += ['blue', 'lightblue']
        x_pos = np.arange(len(labels))
        ax.bar(x_pos, values, color=colors)
        ax.set_xticks(x_pos)
        ax.set_xticklabels(labels, rotation=90)
        yticks = [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi]
        ax.set_yticks(yticks)
        ax.set_yticklabels(['0', 'π/2', 'π', '3π/2', '2π'])
        ax.set_ylim(0, 2*np.pi + 0.75)
        ax.set_ylabel('Angle (radians)', fontsize=14)
        ax.set_title('True and Estimated θ, φ Parameters', fontsize=16)
        plt.tight_layout()
        plt.show()

    def partial_trace_density(self, rho, n_qubits, target_qubit):
        dim = 2**n_qubits
        reduced_rho = np.zeros((2,2), dtype=complex)
        for i in range(dim):
            for j in range(dim):
                i_bits = [(i >> k) & 1 for k in range(n_qubits)][::-1]
                j_bits = [(j >> k) & 1 for k in range(n_qubits)][::-1]
                if all(i_bits[k] == j_bits[k] for k in range(n_qubits) if k != target_qubit):
                    reduced_rho[i_bits[target_qubit], j_bits[target_qubit]] += rho[i,j]
        return reduced_rho

    def bloch_vector_from_rho(self, rho):
        x = 2 * np.real(rho[0,1])
        y = 2 * np.imag(rho[1,0])
        z = np.real(rho[0,0] - rho[1,1])
        return np.array([x, y, z])

    def plot_multi_bloch_qutip(self, input_state, n_qubits):
        if len(input_state.shape) == 1:
            rho = np.outer(input_state, input_state.conj())
        else:
            rho = input_state
        fig, axes = plt.subplots(1, n_qubits, figsize=(4*n_qubits, 4), subplot_kw={'projection': '3d'})
        if n_qubits == 1:
            axes = [axes]
        for qubit in range(n_qubits):
            reduced_rho = self.partial_trace_density(rho, n_qubits, qubit)
            bloch_vec = self.bloch_vector_from_rho(reduced_rho)
            b = Bloch(axes=axes[qubit])
            b.add_vectors(bloch_vec)
            b.render()
            axes[qubit].set_title(f"Qubit {qubit}")
        plt.tight_layout()
        plt.show()

    def run_experiment(self, b):
        with self.output:
            clear_output(wait=True)
            n_qubits = self.n_qubits_slider.value
            n_shotsX = self.n_shotsX_slider.value
            n_shotsY = self.n_shotsY_slider.value
            n_shotsZ = self.n_shotsZ_slider.value

            true_thetas = np.array([slider.value for slider in self.theta_sliders[:n_qubits]])
            true_phis = np.array([slider.value for slider in self.phi_sliders[:n_qubits]])
            true_state = self.generate_multi_qubit_state(true_thetas, true_phis, n_qubits)

            samples = {}
            for setting in itertools.product(['I','X','Y','Z'], repeat=n_qubits):
                if setting != ('I',) * n_qubits:
                    samples[setting] = self.generate_samples(true_state, setting, n_shotsX, n_shotsY, n_shotsZ)

            est_params = self.estimate_parameters(n_qubits, samples)
            est_thetas = est_params[:n_qubits]
            est_phis = est_params[n_qubits:]
            reconstructed_state = self.generate_multi_qubit_state(est_thetas, est_phis, n_qubits)
            fid = self.fidelity(true_state, reconstructed_state)

            print("Fidelity:", fid)
            self.plot_thetas_phis(true_thetas, true_phis, est_thetas, est_phis, fid)
            print("True state Bloch vectors:")
            self.plot_multi_bloch_qutip(true_state, n_qubits)
            print("Reconstructed state Bloch vectors:")
            self.plot_multi_bloch_qutip(reconstructed_state, n_qubits)


In [11]:
app = QuantumTomographyApp()
app.display()
